In [13]:
import importlib
import funciones
importlib.reload(funciones)
from funciones import clean_algabo, clean_furey, concat, clean_canal, scrape_drimel, clean_upalala

def clean_file(canal,filename, company):
    if company == "Algabo":
        clean_algabo(filename)
        clean_canal(canal,'algabo',mpn_value=company)
        concat('./procesadosCanal/algabo.xlsx','./procesados/algabo.xlsx','./listos/algabo.xlsx')
    elif company == "Furey":
        clean_furey(filename)
        clean_canal(canal,'furey',mpn_value=company)
        concat('./procesadosCanal/furey.xlsx','./procesados/furey.xlsx','./listos/furey.xlsx')
    elif company == 'Drimel':
        scrape_drimel(filename)
        clean_canal(canal,'drimel',mpn_value=company)
        concat('./procesadosCanal/drimel.xlsx','./procesados/drimel.xlsx','./listos/drimel.xlsx')
    elif company == 'Upalala':
        clean_upalala(filename)
        clean_canal(canal,'upalala',mpn_value= company)
        concat('./procesadosCanal/upalala.xlsx','./procesados/upalala.xlsx','./listos/upalala.xlsx')
    else:
        raise ValueError("Company not recognized")


In [14]:
# DRIMEL
clean_file('./sinProcesar/canal.csv',
           ['https://drimel.com.ar/?product_cat=bebes', 'https://drimel.com.ar/?product_cat=cuidado-del-bebe'],
            'Drimel')

Archivo guardado con exito
Archivo limpio guardado como ./procesadosCanal/drimel.xlsx
Archivo CONCATENADO guardado como ./listos/drimel.xlsx


In [2]:
# ALGABO
clean_file('./sinProcesar/canal.csv',
           './sinProcesar/algabo.xlsx',
           'Algabo')

/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/funciones.py:146: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Código de barras'] = df['Código de barras'].str.replace('.', '').str.replace('+', '').str.replace('E', '').str.replace('-', '')


Archivo limpio guardado como ./procesados/algabo.xlsx
Archivo limpio guardado como ./procesadosCanal/algabo.xlsx
Archivo CONCATENADO guardado como ./listos/algabo.xlsx


In [8]:
# FUREY
clean_file('./sinProcesar/canal.csv',
           './sinProcesar/furey.xlsx',
           'Furey')

/Users/franciscofurey/00DataScience/Canal/actualizadorPrecios/funciones.py:216: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df = pd.read_excel(filename, dtype={'Código de barras': str})


Archivo limpio guardado como ./procesados/furey.xlsx
Archivo limpio guardado como ./procesadosCanal/furey.xlsx
Archivo CONCATENADO guardado como ./listos/furey.xlsx
